# Зависимости

In [ ]:
import csv
import json
from shortcuts import (read_portion_data, append_functions_in_json, 
                       filter_functions_name, dataset_save_to_csv, 
                       load_functions_from_json)
from settings import FUNCTIONS_PATH, FILTERED_FUNCTIONS_PATH, DATASET_PATH

# Выполнение анализа имен функций

### Подсчёт количества функций

In [ ]:
with open(FUNCTIONS_PATH, 'r') as json_file:
    func_number = 0
    for line in json_file:
        func_number+=1
print('Общее количество функций в датасете:', func_number)

### Подсчет уникальных имен функций и значений узлов

Из-за большого размера файлов оперативная память не справиться с обработкой, поэтому производится порциональный анализ функций.

* `batch_size` - размер порции (1000 для ОЗУ объемом 4 Гб)

* `uniq_names` - словарь уникальных имен функций со стчетчиком

* `uniq_values` - словарь уникальных значений узлов функций

In [ ]:
uniq_names = {}
uniq_values = {}

batch_size = 1000
pos = 0 # начальная позиция чтения файла
counter = 0 # счётчик обработанных функций

# Порционный перебор функций из датасета
functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    counter += len(functions) 
        
    # Добавить имена в словарь подсчета уникальных имен функций
    for func in functions:
        func = json.loads(func)
        uniq_names[func[0]['value']] = uniq_names.get(func[0]['value'], 0) + 1
        # Добавить имена в словарь подсчета уникальных значений узлов
        for node in func:
            if 'value' in node:
                uniq_values[node['value']] = uniq_values.get(node['value'], 0) + 1
            
    #print('Функций обработано: ', counter, '| Ун. имен', len(uniq_names), '| Ун. знач.:', len(uniq_values))

In [ ]:
print(
f'''Функций обработано: {counter}
Уникальных имен функций: {len(uniq_names)}
Уникальных значений узлов: {len(uniq_values)}'''
)
print('Функций потеряно:', func_number-counter)

Импорт/экспорт из/в файл уникальных значений и имен функций.

In [ ]:
with open('datasets/uniq_names.json', 'w') as json_file:
    json.dump(uniq_names, json_file)
with open('datasets/uniq_values.json', 'w') as json_file:
    json.dump(uniq_values, json_file)

In [ ]:
with open('datasets/uniq_names.json', 'r') as json_file:
    uniq_names = json.load(json_file)
with open('datasets/uniq_values.json', 'r') as json_file:
    uniq_values = json.load(json_file)

In [ ]:
bad_keys = []
for i, key in enumerate(uniq_values):
    if '\n' in key:
        bad_keys.append(key)
for bk in bad_keys:
    del uniq_values[bk]

### Выбор самых частотных имен функций

* th - (thresholder) задает порог вхождения имен функциий или значений в выборку

In [ ]:
name_th = 420
value_th = 250

In [ ]:
def cut_dict(dictionary : dict, thresholder : int) -> set:
    data = []
    for key, count in dictionary.items():
        if count > thresholder:
            data.append(key)
    return set(data)

In [ ]:
names = cut_dict(uniq_names, name_th)
values = cut_dict(uniq_values, value_th)

In [ ]:
print('Уникальных имен функций: ', len(names))
print('Уникальных значений узлов функций: ', len(values))

### Фильтрация имен функций по значению

В выборку попадают функции, которые сложно классифицировать по их сниппету, поэтому такие имена функций исключаются из выборки.

In [ ]:
bad_names = ['main', 'run', 'init',
            'iter', 'enter',
            'hash', 'start',
            'nonzero', 'func', 'foo',
             '_run', '__call__', '__enter__',
            '__exit__', '__new__', 'func',
            'fn', 'cb', 'g']
names = set(filter(lambda name: name not in bad_names and len(name) > 2, names))

In [ ]:
print('Уникальных имен функций после чистки на неподходящие имена: ', len(names))

### Фильтрация уникальных значений

Избавление от значений не несущих много информации

In [ ]:
values = set(filter(lambda value: len(value) > 1, values))

In [ ]:
print('Уникальных значений после чистки на неподходящие значения: ', len(values))

## Фильтрация функций по выбранным именам и трансформация узлов

Все функции из файла `functions.json` будут отфильтрованы по имени и записаны в новый файл `filtered_functions.json`.

### Фильтрация по имени

In [ ]:
batch_size = 1000 # Для ОЗУ 8 гигабайт.
pos = 0
filtered_func_number = 0

functions, pos = read_portion_data(FUNCTIONS_PATH, pos, 1)
while functions:
    functions, pos = read_portion_data(FUNCTIONS_PATH, pos, batch_size)
    
    for i,function in enumerate(functions):
        try: # Решает проблему ошибки делиметра
            functions[i] = json.loads(functions[i])
        except json.JSONDecodeError as e:
            print(e)
            functions[i] = functions[i-1]
            continue

    filterd_functions = filter_functions_name(functions, names)
    filtered_func_number += len(filterd_functions)
    
    append_functions_in_json(
        FILTERED_FUNCTIONS_PATH,
        filterd_functions
    )

In [ ]:
print('Количество функций после фильтрации:', filtered_func_number)

### Трансформация узлов

Только для реальных сниппетов

In [ ]:
# Выполнять только при обработке реальных сниппетов
FILTERED_FUNCTIONS_PATH = 'datasets/f.json'
DATASET_PATH = 'datasets/real_snippets.csv'

values = json.load(open('models/dictionary_9875.json', 'r'))
values = list(values.keys())

In [ ]:
functions = load_functions_from_json(FILTERED_FUNCTIONS_PATH)

In [ ]:
import random
random.shuffle(functions)

#### Извлечение названия функции из функции

Удаление из обучающей выборки названия функции и сохранение в отдельном массиве для дальнейшей валидации.

In [ ]:
for func in functions:
    assert func[0]['type'] == 'FunctionDef'

In [ ]:
labels = []
for i, func in enumerate(functions):
    labels.append(functions[i][0]['value'])
    del functions[i][0]

#### Трансформация

In [ ]:
for i,function in enumerate(functions):
    for j,node in enumerate(function):
        try:
            functions[i][j] = node['value'] if node['value'] in values else node['type']
        except KeyError:
            functions[i][j] = node['type']

In [ ]:
print('Количество функций после трансформации:', len(functions))

In [ ]:
assert len(labels) == len(functions)

In [ ]:
print(labels[0], functions[0])

In [ ]:
ave = sum(len(func) for func in functions) / len(functions)
print('Средняя длина вектора АСД: ' + str(ave))

# Сохранение финального датасета

Соединение функции с ее меткой (названием)

In [ ]:
functions_dataset = [(label, func) for (label, func) in zip(labels, functions)]

Сохранение финального датасета для обучения в `.csv` файл

In [ ]:
dataset_save_to_csv(DATASET_PATH, functions_dataset)

In [ ]:
#end